Load a time series of MODIS (MxD11_L2) LST observations from a directory

In [1]:
import xarray as xr
import numpy as np
import os
import pandas as pd
from matplotlib import pyplot as plt
from scipy import interpolate
import cartopy.crs as ccrs
import datetime as dt
import gdal

gdal.UseExceptions()

%matplotlib inline

# Set plot fonts to a larger size
plt.rcParams.update({'font.size': 15})

Find all HDF files in a specified directory

In [2]:
def getListOfFiles(dirName,ext):
    '''Create a list of file names in the given directory with specified file extension.'''
    # https://thispointer.com/python-how-to-get-list-of-files-in-directory-and-sub-directories/
    listOfFile = os.listdir(dirName)
    allFiles = list()
    # Iterate over all the entries
    for entry in listOfFile:
        # Only match files with the correct extension
        if '.'+ext == os.path.splitext(entry)[1]:
            # Create full path and add to list
            fullPath = os.path.join(dirName, entry)
            allFiles.append(fullPath)       
    return allFiles

In [3]:
storage = r'//j-lundquist-3.ce.washington.edu/storage'

lst_searchDir = storage + r'/MODIS/Tuolumne_MxD11_L2/2017/'
lst_file_list = getListOfFiles(lst_searchDir,'hdf')

geo_searchDir = storage + r'/MODIS/Tuolumne_MxD03/2017/'
geo_file_list = getListOfFiles(geo_searchDir,'hdf')

print('Found {} LST files, {} GEO files'.format(len(lst_file_list),len(geo_file_list)))

Found 96 LST files, 83 GEO files


Open files and extract the LST, view angle, latitude, and longitude subdatasets.

In [4]:
lst_ds = [gdal.Open('HDF4_EOS:EOS_SWATH:"{}":MOD_Swath_LST:LST'.format(path)) for path in lst_file_list]
viewangle_ds = [gdal.Open('HDF4_EOS:EOS_SWATH:"{}":MOD_Swath_LST:View_angle'.format(path)) for path in lst_file_list]
geo_lat_ds = [gdal.Open('HDF4_SDS:UNKNOWN:"{}":0'.format(path)) for path in geo_file_list]
geo_lon_ds = [gdal.Open('HDF4_SDS:UNKNOWN:"{}":1'.format(path)) for path in geo_file_list]

Create empty arrays to stack data into.

In [5]:
n_files = len(lst_ds)
m_files = len(geo_lat_ds)
# Because the MODIS products are either 2030 or 2040 pixels in 5 minutes...
# I'm truncating them all to just 2030
along_track_px = 2030
cross_track_px = 1354

# Create empty arrays to hold the radiance, lat, and lon

lst = np.ones((along_track_px,  # Width
               cross_track_px), # Height
              dtype='float64')

viewangle = np.ones((along_track_px,  # Width
               cross_track_px), # Height
              dtype='float64')

lon = np.ones((along_track_px,  # Width
               cross_track_px), # Height
              dtype='float64')

lat = np.ones((along_track_px,  # Width
               cross_track_px), # Height
              dtype='float64')

Match geolocation with LST products, stack into xarray dataset.

In [ ]:
# LST scale factor
lst_scale_factor=0.02
# View angle scale factor
view_scale_factor = 0.5
# For each file in the timeseries of MODIS observations
k = 0
for i in range(0,n_files):
    # Read the date and time from the LST product
    date = lst_ds[i].GetMetadataItem('RANGEBEGINNINGDATE')
    time = lst_ds[i].GetMetadataItem('RANGEBEGINNINGTIME')
    eq_cross_time = lst_ds[i].GetMetadataItem('EQUATORCROSSINGTIME.1')
    # Find its matching MxD03 Geolocation product
    for j in range(0,m_files):
        # date_geo = geo_lat_ds[j].GetMetadataItem('RANGEBEGINNINGDATE')
        # time_geo = geo_lat_ds[j].GetMetadataItem('RANGEBEGINNINGTIME')
        eq_cross_time_geo = geo_lat_ds[j].GetMetadataItem('EQUATORCROSSINGTIME.1')
        # Once we find a match, load these into our arrays
        if eq_cross_time==eq_cross_time_geo:# and lst_ds[i].ReadAsArray() is not None:
            print('Found files for {} {}'.format(date,time))
            # Load the LST values and scale them
            lst = lst_scale_factor * lst_ds[i].ReadAsArray()[0:along_track_px,0:cross_track_px]
            # Replace the nodata value 0, with Nans
            lst[lst==0.] = np.nan
            # Read the view angles from the LST product, scale and remove nodata values
            viewangle = viewangle_ds[i].ReadAsArray()[0:along_track_px,0:cross_track_px].astype(float)
            viewangle[viewangle==255] = np.nan
            viewangle = view_scale_factor * viewangle
            # Read the latitudes and longitudes from the geolocaiton product
            lat = geo_lat_ds[j].ReadAsArray()[0:along_track_px,0:cross_track_px]
            lon = geo_lon_ds[j].ReadAsArray()[0:along_track_px,0:cross_track_px]
            # Add the MODIS data to an xarray dataset
            if k==0: # If this is our first file, make a new dataset
                ds = xr.Dataset({'lst': (['line', 'pixel'],  lst),
                                'viewangle': (['line', 'pixel'],  viewangle)},
                                coords={'longitude': (['line', 'pixel'], lon),
                                        'latitude': (['line', 'pixel'], lat),
                                        'time': pd.to_datetime('{} {}'.format(date,time))})
                k+=1 # add to the successful file load counter
            else: # append to existing dataset
                ds_new = xr.Dataset({'lst': (['line', 'pixel'],  lst),
                                'viewangle': (['line', 'pixel'],  viewangle)},
                                coords={'longitude': (['line', 'pixel'], lon),
                                        'latitude': (['line', 'pixel'], lat),
                                        'time': pd.to_datetime('{} {}'.format(date,time))})
                ds = xr.concat([ds,ds_new],'time') # concatenate along the time axis
                k+=1 # add to the successful file load counter
            break # go to next LST file

# Finally sort by time:
ds = ds.sortby(ds.time)
print('{} out of {} files loaded with geolocation information'.format(k,n_files))

# Update to the number of files we successfully loaded
n_files = ds.time.shape[0]

Export this stack of MODIS observations as a new NetCDF file

In [ ]:
ds

In [ ]:
ds.to_netcdf('./nc/tuolumne-201704151925-201704302200-MxD11_L2-MxD03.nc',mode='w')

---
work in progress...

Subset to some bounding box around our site of interest:

In [30]:
# Set up the coordinates of a bounding box around our site of interest

lat_obs = 37.88 
long_obs = -119.31

n_bound = lat_obs + 1
s_bound = lat_obs - 1
e_bound = long_obs + 1
w_bound = long_obs - 1

In [31]:
# Trim the datasets to only within this bounding box

ds2 = ds.where( (ds.latitude>=s_bound) & 
               (ds.latitude<=n_bound) & 
               (ds.longitude>=w_bound) & 
               (ds.longitude<=e_bound), 
               drop=True)

In [32]:
## Plot all the resulting images from this bounding box
#
#for i in range(ds2.time.size):
#    ds2.lst[i].plot(x='longitude',y='latitude',add_colorbar=False)
#    plt.xlim((w_bound,e_bound))
#    plt.ylim((s_bound,n_bound))
#

Export this stack of MODIS observations as a new NetCDF file

In [33]:
ds2

<xarray.Dataset>
Dimensions:    (line: 2030, pixel: 1354, time: 83)
Coordinates:
    longitude  (time, line, pixel) float32 -127.85866 -127.798355 ... -150.81458
    latitude   (time, line, pixel) float32 55.418823 55.41911 ... 44.437504
  * time       (time) datetime64[ns] 2017-04-10T09:40:00 ... 2017-04-30T22:00:00
Dimensions without coordinates: line, pixel
Data variables:
    lst        (time, line, pixel) float64 nan nan nan nan ... nan nan nan nan
    viewangle  (time, line, pixel) float64 nan nan nan nan ... nan nan nan nan

In [11]:
ds2.to_netcdf('./nc/tuolumne_bbox-201704151925-201704302200-MxD11_L2-MxD03.nc',mode='w')

***
***